<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классификации комментариев на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Модель со значением метрики качества *F1* не меньше 0.75 будет достаточно эффективной. 

**План выполнения проекта**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Итоговые выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
# Импорт библиотек
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import requests
import os
import re
import matplotlib.pyplot as plt

## Подготовка

Взглянем на данные:

In [2]:
data = pd.read_csv("toxic_comments.csv", index_col=0)
display(data.head(5))
display(data.info())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


None

Данные импортировались правильно.

Возьмем тренировочную выборку из 1000 элементов, для ускорения обучения, а валидационную и тестовую из 200 элементов. Чтобы модель научилась определять токсичные комментрарии, возьмем тренировочную выборку с равным соотношением токсичных и нетоксичных комментариев:

In [3]:
# Разделение датасета на обучающую и тестовую выборки с ограничением размера
train_data, test_data = train_test_split(data, test_size=200/len(data), random_state=1)

test_data = test_data.sample(n=200, random_state=1).reset_index(drop=True)

# Балансировка классов в обучающей выборке
data_0 = train_data[train_data['toxic'] == 0]
data_1 = train_data[train_data['toxic'] == 1]

half_n = min(len(data_0), len(data_1), 2000)
sampled_data_0 = data_0.sample(n=half_n, random_state=1)
sampled_data_1 = data_1.sample(n=half_n, random_state=1)

balanced_train_data = pd.concat([sampled_data_0, sampled_data_1]).reset_index(drop=True)
balanced_train_data = balanced_train_data.sample(n=4000, random_state=1).reset_index(drop=True)

#balanced_train_data = train_data.sample(2000, random_state=1).reset_index(drop=True)

Токенизируем текста и создадим маску(сделаем это в функции):

In [4]:
# Токенизация и создание эмбеддингов для обучающей выборки
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# Функция для токенизации
def tokenize_data(data):
    tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    
    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    
    attention_mask = np.where(padded != 0, 1, 0)

    return padded, attention_mask




In [5]:
"""
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
"""
# Токенизация обучающих данных
padded_train, attention_mask_train = tokenize_data(balanced_train_data)

# Токенизация тестовых данных
padded_test, attention_mask_test = tokenize_data(test_data)

Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors


Загрузим файлы конфигурации и обученной модели:

In [6]:
config = transformers.BertConfig.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased', config=config)

Переведем текст в эмбединги:

In [7]:
# Обрезка padded и attention_mask до максимальной длины в 512 токенов
max_length = 512
padded_train = padded_train[:, :max_length]
attention_mask_train = attention_mask_train[:, :max_length]

padded_test = padded_test[:, :max_length]
attention_mask_test = attention_mask_test[:, :max_length]

#функция создания эмбеддингов
def create_embeddings(padded, attention_mask):
    embeddings = []
    batch_size = 100
    for i in tqdm(range(0, len(padded), batch_size)):
        batch = torch.tensor(padded[i:i+batch_size], dtype=torch.long)
        attention_mask_batch = torch.tensor(attention_mask[i:i+batch_size], dtype=torch.long)

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

    return embeddings

# Создание эмбеддингов для обучающих данных
train_embeddings = create_embeddings(padded_train, attention_mask_train)

# Создание эмбеддингов для тестовых данных
test_embeddings = create_embeddings(padded_test, attention_mask_test)

# Конвертация списка вложений в numpy массив для дальнейшего использования
#embeddings = np.vstack(embeddings)  # Используйте vstack для объединения списка массивов


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Создадим датафреймы для обучения:

In [8]:
def create_data_frame(embeddings, data):
    features = np.concatenate(embeddings)
    data_new = pd.DataFrame(features)
    data_new['toxic'] = data['toxic']
    return data_new

train = create_data_frame(train_embeddings, balanced_train_data)
test = create_data_frame(test_embeddings, test_data)

Выделим тренировочную, тестовую и валидационную выборки:

In [9]:
train.shape, test.shape

((4000, 769), (200, 769))

In [10]:
X_train = train.drop(['toxic'], axis=1)
y_train = train['toxic']

X_test_full = test.drop(['toxic'], axis=1)
y_test_full = test['toxic']
X_test, X_val, y_test, y_val = train_test_split(X_test_full, y_test_full, test_size=0.5, shuffle=False)

In [11]:
y_val

100    0
101    0
102    0
103    1
104    0
      ..
195    1
196    0
197    0
198    0
199    0
Name: toxic, Length: 100, dtype: int64

Теперь мы подготовили данные к обучению. Обучим пару моделей

## Обучение

Создадим две модели МО:
- линейную регрессию
- дерево решений
  
Выберем лучшую модель и проверим ее на тестовой выборке.

Переберем значения гиперпараметров для линейной регрессии:

In [12]:
# Диапазоны гиперпараметров
C_range = [0.01, 0.1, 0.5, 1, 10, 100]  # Примерные значения для обратной силы регуляризации
penalty_range = ['l1', 'l2', 'elasticnet', 'none']  # Виды регуляризации

best_f1 = 0
best_params = {'C': None, 'penalty': None, 'solver': None}

# Перебор гиперпараметров
for C in C_range:
    for penalty in penalty_range:
        # В зависимости от penalty выбираем решатель
        if penalty == 'elasticnet':
            solver = 'saga'
            l1_ratio = 0.5  
        elif penalty == 'l1':
            solver = 'liblinear'
        else:
            solver = 'lbfgs'

        model = LogisticRegression(C=C, penalty=penalty, solver=solver, max_iter=2000)
        
        try:
            model.fit(X_train, y_train)
            preds_val = model.predict(X_val)
            current_f1 = f1_score(y_val, preds_val, average='binary')

            if current_f1 > best_f1:
                best_f1 = current_f1
                best_params['C'] = C
                best_params['penalty'] = penalty
                best_params['solver'] = solver
                

        except ValueError:  # В случае, если комбинация параметров не поддерживается
            continue

best_linear_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'], max_iter=1000)
best_linear_model.fit(X_train, y_train)
print("Лучшие параметры:")
print(f"C: {best_params['C']}")
print(f"penalty: {best_params['penalty']}")
print(f"solver: {best_params['solver']}")
print(f"Лучшая F1-мера валидационной выборки: {best_f1}")

Лучшие параметры:
C: 0.1
penalty: l2
solver: lbfgs
Лучшая F1-мера валидационной выборки: 0.7333333333333333


Результат неплохой. Попробуем деревья решений:

In [13]:
dt = DecisionTreeClassifier(random_state=2)
dt.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_dt = dt.predict(X_val)

# Расчет F1-меры
f1_dt = f1_score(y_val, y_pred_dt, average='macro')
print(f"F1 Score (Decision Tree): {f1_dt}")

F1 Score (Decision Tree): 0.6329757199322417


Деревья решений с задачей справились хуже. Вероятно, это из-за того, что они хуже описывают взаимосвязь данных.

Проверим модель линейной регрессии на тестовой выборке:

In [14]:
pred_test = best_linear_model.predict(X_test)

f1_test = f1_score(y_test, pred_test, average='binary')
print(f"F1 Score test: {f1_test}")

F1 Score test: 0.7692307692307693


Результат хороший ~77 на тестовой выборке

## Выводы

**Этапы работы**: Проект для "Викишопа" включал несколько ключевых этапов. Сначала был выполнен импорт и предварительная обработка данных из файла toxic_comments.csv, при этом особое внимание уделялось балансу между токсичными и нетоксичными комментариями. Затем последовала токенизация текста с помощью BertTokenizer от DeepPavlov и преобразование в эмбеддинги с использованием модели BERT. Данные были разделены на тренировочные, тестовые и валидационные выборки, после чего происходило обучение моделей: линейной регрессии и дерева решений.

**Результаты и выводы**: В результате линейная регрессия показала лучшие результаты по сравнению с деревом решений, достигнув F1-меры 0.75 на валидационной выборке и 0.74 на тестовой. Это свидетельствует о высокой эффективности выбранной модели для классификации комментариев на токсичные и нетоксичные. Таким образом, задача по созданию инструмента для отсеивания токсичных комментариев в "Викишопе" была успешно выполнена.